# Puzzle 8: Seven Segment Search

This puzzle seems much more challenging than those that came before.

In [138]:
filename = "input8.txt"
with open(filename,"r") as infile:
    input = [line.strip().split("|") for line in infile]

print(input[0])

['eadbcf faceb faecgd gdefabc adc ad adbf gfacbe bceda dcegb ', ' gdfcae adc cedbfa dafb']


In [139]:
signalPatterns = [pattern[0].split() for pattern in input]
outputValues = [value[1].split() for value in input]
combinedData = list(zip(signalPatterns, outputValues))
print(signalPatterns[0], outputValues[0])
print(combinedData[0])

['eadbcf', 'faceb', 'faecgd', 'gdefabc', 'adc', 'ad', 'adbf', 'gfacbe', 'bceda', 'dcegb'] ['gdfcae', 'adc', 'cedbfa', 'dafb']
(['eadbcf', 'faceb', 'faecgd', 'gdefabc', 'adc', 'ad', 'adbf', 'gfacbe', 'bceda', 'dcegb'], ['gdfcae', 'adc', 'cedbfa', 'dafb'])


### Part 1
The seven segment displays of 1   4   7   8, are unique in that they each are the only values to use 2,4,3,7 segments to display their value on the seven segment display.

In [140]:
# count each of the occurences of these values in output values (method 1)
count = 0
for value in combinedData[0][1]:
    if len(value) in (2,3,4,7):
        print(value, len(value))
        count += 1
count

adc 3
dafb 4


2

In [141]:
# method 2, use a dictionary (or map as its called cpp)

# convert the list of output values to a flat list
oneDimensionalOutputvalues = [value for row in outputValues for value in row]
# list of the lengths of each output value using map func
lengthsOfOutputValues = list(map(lambda x: len(x), oneDimensionalOutputvalues))

# count the total occurences of each length (so occurences of unique seven segment digits)
from collections import Counter as counter
countsOfLengths = counter(lengthsOfOutputValues) # {}

total = 0
for k,v in countsOfLengths.items():
    if k in (2,3,4,7): # the unique digits
        total += v
        # print(k, v)
print(f"In the output values, the digits 1, 4, 7, or 8 appear a total of <{total}> times") # answer to part 1


In the output values, the digits 1, 4, 7, or 8 appear a total of <539> times


### Part 2
Determine the remaining digits

In [142]:
#   0:     
#  aaaa    
# b    c  
# b    c  
#  dddd   
# e    f  
# e    f  
#  gggg 

# load in the example from question
testSignalPatterns = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab'.split()
testOutputValues = "cdfeb fcadb cdfeb cdbaf".split()
print(testSignalPatterns, testOutputValues, sep="\n\n")

['acedgfb', 'cdfbe', 'gcdfa', 'fbcad', 'dab', 'cefabd', 'cdfgeb', 'eafb', 'cagedb', 'ab']

['cdfeb', 'fcadb', 'cdfeb', 'cdbaf']


Puzzle input consists of essentially line after line of the same type of scrambled codes. Each one is scrambled similarly as described in the puzzle description, but each line is a unique 'cypher', if it can be called that.

Solved the example problem in the question, on paper. Next step is to design a set of functions to do this, so it can be performed quickly on every single line of the puzzle input.

In [143]:
lettersList = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
signalMapping = dict(list(zip(lettersList,[None]*10)))
print(signalMapping)

{'a': None, 'b': None, 'c': None, 'd': None, 'e': None, 'f': None, 'g': None}


In [144]:
# possibly remove decode approach and reuse signal mapping variable name for number translated

# finds the encoded numbers (1,4,8,7) by looking at their unique lenghts of (2,4,7,3) respectively
# returns dictionary of numbers (keys) and encoded strings (values)
def find_unique_lengths(signalPatterns):
    numbersTranslated = dict(list(zip(list(range(0,10)),[None]*10)))
    uniqueLengths = {2:1, 4:4, 7:8, 3:7} # {length:integer represented}

    for item in signalPatterns:
        if len(item) in uniqueLengths.keys():
            numbersTranslated[uniqueLengths[len(item)]] = item

    return numbersTranslated

numbersTranslated = find_unique_lengths(testSignalPatterns)
numbersTranslated

{0: None,
 1: 'ab',
 2: None,
 3: None,
 4: 'eafb',
 5: None,
 6: None,
 7: 'dab',
 8: 'acedgfb',
 9: None}

The variable numbers translated will be used to compare to the output values to find out what integer they represent, in the puzzle input.

In [145]:
# find the top segment signal code
top = [i for i in numbersTranslated[7] if i not in numbersTranslated[1]][0]
top

'd'

Integer 1 is important as it contains only UR and LR segments. We can look at the three different signals with six segments. The pattern that does not contain both signals in the signal for 1, must be 6. This is because the three integers with 6 segments are 0, 6, and 9. 0 and 9 both incorporate the same segments as 1. 

In [146]:
def find_signals_length_6(signalPatterns):
    return [s for s in signalPatterns if len(s) == 6]

def find_signals_length_5(signalPatterns):
    return [list(signal) for signal in signalPatterns if len(signal) == 5]

# Find all integers that use 6 segments, by filtering signalPatterns for len(6)
def find_pattern_for_six(signalPatterns: list, numbersTranslated: dict ):
    one = numbersTranslated[1]
    signalPatternsLengthSix = find_signals_length_6(signalPatterns)
    six = [s for s in signalPatternsLengthSix if one not in s][0]
    # both the list comprehensions evaluate to list of length 1
    UR = [s for s in list(one) if s not in list(six)][0]
    BR = list(filter(lambda x: x != UR, list(one)))[0]

    return six, UR, BR

six, UR, BR = find_pattern_for_six(testSignalPatterns, numbersTranslated)
# add pattern for six to numberTranslated
numbersTranslated[6] = six

print(numbersTranslated)

{0: None, 1: 'ab', 2: None, 3: None, 4: 'eafb', 5: None, 6: 'cdfgeb', 7: 'dab', 8: 'acedgfb', 9: None}


Find the middle segment, which allows us to distinguish between 0 and 9 integer signal patterns. We do this by looking at the remaining length 6 integer signal patterns and finding the signal characters (a ~ g, excluding the known characters), of which there are three. We will see that there are only two signal characters that differ between the remaining letters for each signal patterns. One of these letters will be present in all of the signal patterns of length five, that is integers 2,3,5. 
This signal is for the middle segment and allows us to determine the signal patterns for 0 and 9.

As a consequence of this we will know the signal pattern of 9 since in the initial set of patterns of length 6, it is the only one to lack the BL segment. Thus 0 will be the other pattern. 

In [154]:
# removes known integer 6 from the signal patterns of length six filtered list
def filter_int6_spl6(signalPatternsLength6, numbersTranslated):
    # extract relevent known values
    six, seven = numbersTranslated[6], numbersTranslated[7]
    # remove the known singal pattern for integer 6
    signalPatternsLength6 = [list(item) for item in signalPatternsLength6 if item != six]
    # create new list of with these signal patterns excluding the know signal characters
    return [list(filter(lambda x: x not in seven, item)) for item in signalPatternsLength6] # spl6

# finds the middle segment signal from comparsion of spl6 and spl5
def find_middle_vals(spl5, spl6):
    # creates a list of element not common between spl5 and spl6 lists
    potentialMiddleVals = list(set(spl6[0]).difference(spl6[1])) + list(set(spl6[1]).difference(spl6[0]))
    return [s for s in potentialMiddleVals if s in spl5[0] and spl5[1] and spl5[2]][0] # middle segment signal

def find_zero_and_nine(signalPatterns, numbersTranslated): 
    # create list of signal patterns length 5
    spl5 =  find_signals_length_5(signalPatterns)

    signalPatternsLength6 = find_signals_length_6(signalPatterns)
    spl6 = filter_int6_spl6(signalPatternsLength6, numbersTranslated)
    
    middle = find_middle_vals(spl5,spl6)

    for i,signal in enumerate(signalPatternsLength6):
        if middle in signal:
            nine = ''.join(signal)
            zero = ''.join(signalPatternsLength6[i+1])
        else:
            nine = ''.join(signalPatternsLength6[i+1])
            zero = ''.join(signal)
        break
    return nine, zero, middle

nine, zero, middle = find_zero_and_nine(testSignalPatterns, numbersTranslated)
numbersTranslated[9], numbersTranslated[0] = nine, zero
print(numbersTranslated)


{0: 'cdfgeb', 1: 'ab', 2: None, 3: None, 4: 'eafb', 5: None, 6: 'cdfgeb', 7: 'dab', 8: 'acedgfb', 9: 'cefabd'}


Now to examine signal pattern for int 4, to determine the UL (upper left) signal character

In [162]:
def find_UL(four, middle, UR, BR):
    UL = [item for item in list(four) if item not in middle or UR or BR][0]
    return UL # Upper left UL

UL = find_UL(numbersTranslated[4], middle, UR, BR)
print(UL)


e


We can now find the signal patterns associated with the integers 2,3,5.

In [178]:
def find_five(signalPatterns, UL):
    spl5 = find_signals_length_5(signalPatterns)
    spl5 = [''.join(s) for s in spl5] # make each signal pattern a string of chars
    # only one element of spl5 will contain the UL character
    return [s for s in spl5 if UL in s][0]

five = find_five(testSignalPatterns,UL)
numbersTranslated[5] = five
print(numbersTranslated)

{0: 'cdfgeb', 1: 'ab', 2: None, 3: None, 4: 'eafb', 5: 'cdfbe', 6: 'cdfgeb', 7: 'dab', 8: 'acedgfb', 9: 'cefabd'}


We can now easily find the remaining integers' (2,3) signal patterns

In [202]:
def find_two_and_three(signalPatterns, numbersTranslated):
    spl5 = find_signals_length_5(signalPatterns)
    spl5.remove(list(numbersTranslated[5]))
    spl5 = [''.join(s) for s in spl5]

    for i, signal in enumerate(spl5):
        if numbersTranslated[1][0] and numbersTranslated[1][1] in signal:
            three = signal
            spl5.remove(three)
            two = spl5[0]
    return two, three
    
two, three = find_two_and_three(testSignalPatterns, numbersTranslated)
numbersTranslated[2] = two
numbersTranslated[3] = three
print(numbersTranslated)

{0: 'cdfgeb', 1: 'ab', 2: 'gcdfa', 3: 'fbcad', 4: 'eafb', 5: 'cdfbe', 6: 'cdfgeb', 7: 'dab', 8: 'acedgfb', 9: 'cefabd'}


find the 

### Main Loop
This demonstrates the process on an example, this loop does the same for every single set of signal patterns in the puzzle input.

In [ ]:
def

The functions below are called on the output values after the | demlimiter

In [ ]:
# # for testing
# encodedlist = ['d', 'e', 'a', 'f', 'g', 'b', 'c']
# signalMapping = dict(list(zip(lettersList,encodedlist))
# encodedNumberTest = 'cdfeb' # => abdfg => 5

In [ ]:
# may not need to use this

# takes an encoded number string and returns decodedNumber as list in alphabetical order
def decode_letters_string(encodedNumber: str, signalMapping: dict):
    return ''.join(sorted([signalMapping[letter] for letter in list(encodedNumber)]))

# decodedNumber = decode_letters_string(encodedNumberTest, signalMapping)

In [ ]:
# takes string input decoded number which is a signal to the display
# and returns the integer value it represents
def translate_2_seven_segment_num(decodedNumber):
    NormalisedSignalsList = ['abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg']
    lettersTranslateDict = dict(list(zip(NormalisedSignalsList,list(range(0,10)))))

    if decodedNumber in lettersTranslateDict.keys():
        return lettersTranslateDict[decodedNumber]
    else:
        ValueError

# translate_2_seven_segment_num(decodedNumber)